In [149]:
import pandas as pd
import numpy as np

In [150]:
df = pd.read_csv("DonneesMeteo.txt",delim_whitespace=True, low_memory=False, skiprows=1)

In [151]:
import datetime
#time = temp = pd.DatetimeIndex(df1['DT_local'])
date = pd.DatetimeIndex(pd.to_datetime(df['Date'], format="%Y%m%d"))
df['MONTH'] = date.month
df['DAY'] = date.day
df['YEAR'] = date.year
df['DOY'] = date.dayofyear
#Version arrondie à la minute de la ligne du dessous : num = pd.to_datetime(pd.Series.round(df['hh(UTC)']*3600)*1000000000)
hh = pd.DatetimeIndex(pd.to_datetime(df['hh(UTC)']*3600*1000000000))
df['HOUR'] = hh.hour
df['MINUTE'] = hh.minute

In [153]:
#conversion des heures de la journee

On va à présent se concentrer sur la prédiction de :
    - puissance
Selon
    - T
    - vent
    - skycover
    - precipitation

In [154]:
#df.columns.values.tolist()

# RandomForest

In [279]:
#Paramètres
'''colsX = ['DAY',
    'Date',
 'hh(UTC)',
 'Tp1(C)',
 'Tp2(C)',
 'IrrPOA(W/m2)',
 'Gz2(W/m2)',
 'Drz2(W/m2)',
 'Dfz2(W/m2)',
 'IRz2(W/m2)',
 'T(C)',
 'WS(m/s)',
 'WD(deg)',
 'mpGz1(W/m2)',
 'IRz1dn(W/m2)',
 'IRz1up(W/m2)',
 'mpT(C)',
 'mpWS(m/s)',
 'mpWD(deg)',
 'SZA(deg)',
 'SAA(deg)']'''
colsX = ["DOY", "hh(UTC)", "IrrPOA(W/m2)"]
colsY = ["P1(W)", "P2(W)"]
nDOY = 0
nhh = 1
#print(cols)

In [188]:
import sklearn.cross_validation as cv
import sklearn.preprocessing as pp #pour les scaling

In [189]:
def prepare(df,colsX,colsY):
    #Split the treated data between X and y
    Z = vireNan(df[colsX+colsY])
    deb = 0
    sep = len(colsX)#separteur entre les X et les Y
    fin = sep+len(colsY)
    X=Z[:,deb:sep]
    y=Z[:,sep:fin]
    return X,y

import numpy as np
def vireNan(Z) :
    # Load the dataset
    Z = np.array(Z).astype(np.float)

    #Remove nan and infinite values
    masknan = ~np.any(np.isnan(Z), axis=1)
    Z = Z[masknan]
    maskfin = np.any(np.isfinite(Z), axis=1)
    Z = Z[maskfin]
    return Z

In [190]:
#preparer (nan, découpe)
X,y = prepare(df,colsX,colsY)

In [191]:
def decoupe(X,y) :
    X_train, X_1, y_train, y_1 = cv.train_test_split(X, y, test_size=0.4, random_state=0)
    X_cv, X_test, y_cv, y_test = cv.train_test_split(X_1, y_1, test_size=0.1, random_state=0)
    return X_train, X_cv, X_test, y_train, y_cv, y_test

In [192]:
#Découper
X_train, X_cv, X_test, y_train, y_cv, y_test = decoupe(X,y)

In [193]:
#variable globales
fsX = pp.StandardScaler()
fsy = pp.StandardScaler()
def featureScale(X_train, X_cv, X_test, y_train, y_cv, y_test) :
    #feature scaling inti
    fsX.fit(X_train)
    X_train = fsX.transform(X_train)
    X_cv = fsX.transform(X_cv)
    X_test = fsX.transform(X_test)
    
    fsy.fit(y_train)
    y_train = fsy.transform(y_train)
    y_cv = fsy.transform(y_cv)
    y_test = fsy.transform(y_test)
    
    return X_train, X_cv, X_test, y_train, y_cv, y_test

In [194]:
#scaler par rapport au train
X_train, X_cv, X_test, y_train, y_cv, y_test = featureScale(X_train, X_cv, X_test, y_train, y_cv, y_test)

In [225]:
from sklearn.ensemble import RandomForestRegressor

def rms(Y1,Y2):
    return np.sqrt(np.mean((Y1 - Y2)**2))

def randomforest(n,X,Y,X_test):
    clf= RandomForestRegressor(n_estimators=n)
    clf.fit(X,Y)
    y_=clf.predict(X_test)
    return y_

In [226]:
def randomforestcrossovern(nArray, X_train, X_cv, y_train, y_cv):
    #Conversion
    nArray = np.array(nArray).astype(np.integer)
    #tableau avec les valeurs d'erreur
    RMS=np.zeros((nArray.size,2))
    
    i=0
    for n in nArray:
        y_=randomforest(n,X_train,y_train,X_cv)
        r =rms(y_,y_cv)
        RMS[i,0]=n
        RMS[i,1]=r
        i+=1
    plt.scatter(RMS[:,0],RMS[:,1])
    plt.show()
    
    return RMS

In [227]:
#Pour plusieurs n ? on prends n =30, suffisant, erreur de 0.2
nArray = [1,3,10,30,50,80,100,200,300]
#Cross sur les nArray
randomforestcrossovern(nArray, X_train, X_cv, y_train, y_cv)

array([[  1.00000000e+00,   2.91105504e-01],
       [  3.00000000e+00,   2.35810442e-01],
       [  1.00000000e+01,   2.16882987e-01],
       [  3.00000000e+01,   2.06642553e-01],
       [  5.00000000e+01,   2.06300541e-01],
       [  8.00000000e+01,   2.05409365e-01],
       [  1.00000000e+02,   2.05622288e-01],
       [  2.00000000e+02,   2.03638387e-01],
       [  3.00000000e+02,   2.03139790e-01]])

In [291]:
def convertirExtensif(deb,fin,pas,X):
    nCols = int((fin-deb)/pas)
    m = np.shape(X)[0]
    deb = deb * np.ones((m))#pour soustraction
    y = np.zeros((m,nCols))
    #print("nCols",nCols)
    
    #conversion pour les index
    idxArray = np.divide((X-deb),pas)
    #virer les cas ou egal a fin
    maxVal = (fin-deb)/pas
    idxArray = idxArray - (idxArray >= maxVal)*maxVal
    #print(X)
    #print("idxArray",idxArray)
    
    l = 0
    for idx in idxArray:
        y[l,idx] = 1
        l+=1
    #print(y)
    return y

def convertirExtensifEnPlace(nCol,deb,fin,pas,X):
    return np.append( np.delete(X, (nCol), axis=1) , convertirExtensif(deb,fin,pas,X[:,nCol]) , (1))#(1) pour l'axe "col"
    

def randomforestcrossoverext(nCol,deb,fin,pasArray,n, X_train, X_cv, y_train, y_cv):
    pasArray = np.array(pasArray)
    RMS=np.zeros((pasArray.size,2))
    
    i=0
    for pas in pasArray:
        #print("")
        print("pas",pas)
        #print("shape X_train",np.shape(X_train))
        #Transformation data
        X_train_ =convertirExtensifEnPlace(nCol,deb,fin,pas,X_train)
        X_cv_    =convertirExtensifEnPlace(nCol,deb,fin,pas,X_cv)
        #print(np.shape(X_train_))
        #print(np.shape(X_cv_))
        #print(np.shape(y_train))
        y_= randomforest(n,X_train_,y_train,X_cv_)
        r = rms(y_,y_cv)
        RMS[i,0]=pas
        RMS[i,1]=r
        i+=1
    plt.scatter(RMS[:,0],RMS[:,1])
    plt.show()
    
    return RMS

In [292]:
n=30
#Cross sur les differentes extensions
#d'abord DOY
deb =0
fin =366
pas = [1,10,30,60,100,150]
randomforestcrossoverext(nDOY,deb,fin,pas,n, X_train, X_cv, y_train, y_cv)

pas 1
pas 10
pas 30
pas 60
pas 100
pas 150


array([[   1.        ,    0.24391008],
       [  10.        ,    0.26235255],
       [  30.        ,    0.26027202],
       [  60.        ,    0.26280532],
       [ 100.        ,    0.26227857],
       [ 150.        ,    0.26056541]])

In [288]:
n=30
#Cross sur les differentes extensions
#d'abord DOY
deb =0
fin =24
pas = [0.1,0.4,7.8,1,3,6,9,12]
randomforestcrossoverext(nHOUR,deb,fin,pas,n, X_train, X_cv, y_train, y_cv)

pas 0.1
pas 0.4
pas 7.8
pas 1.0
pas 3.0
pas 6.0
pas 9.0
pas 12.0


array([[  0.1       ,   0.22022589],
       [  0.4       ,   0.22352416],
       [  7.8       ,   0.23874988],
       [  1.        ,   0.23052719],
       [  3.        ,   0.24110565],
       [  6.        ,   0.2399354 ],
       [  9.        ,   0.24201735],
       [ 12.        ,   0.24021124]])

In [181]:
n=1
a = np.array([[1,2,3],[4,5,6]])
print(a)
print(np.delete(a,(n),axis=1))

[[1 2 3]
 [4 5 6]]
[[1 3]
 [4 6]]


In [ ]:
plotterIterations(X_test,1,0,y_test,0)

In [7]:
# Plot outputs 2D
import matplotlib.cm as cm

def plotterIterations(X,nX,nIt,y,ny) :
    #On se ramène à des journées
    EltIteration = X[:,nIt]
    Iterations = np.unique(EltIteration)
    print(Iterations)

    #Gérer plusieurs couleurs
    colors = cm.rainbow(np.linspace(0, 1, len(Iterations)))

    for k,c in zip(Iterations,colors)  :
        #On prend que le jour
        condition = EltIteration == k #entre 60 et 153
        xPlot = np.extract(condition, X[:,nX])
        yPlot = np.extract(condition, y[:,ny])
        plt.scatter(xPlot,yPlot, color=c,s=2)
        #sort
        #print(np.concatenate(xPlot,yPlot))
        #z = np.sort(np.concatenate(xPlot,yPlot),0)
        #plt.plot(z[:,0], z[:,1], color=c,linewidth=2)
        #print(k)
        #print(xPlot)
        #print(yPlot)
        #print(z[:,0])
        #print(z[:,1])
        #break

    #plt.plot(X_test, regr.predict(X_test), color='blue',linewidth=3)
    plt.xticks(())
    plt.yticks(())

    plt.title('')

    plt.show()